# SageMaker Real-Time Deployment (GTZAN CNN)

This notebook packages model artifacts, uploads to S3, and deploys a real-time endpoint using `inference.py`.


In [ ]:
# Update these paths if needed
from pathlib import Path

MODEL_SRC = Path('..') / 'models'
PROCESSED_DIR = Path('..') / 'data' / 'processed'
MODEL_TAR = Path('model.tar.gz')


In [ ]:
# Package model artifacts into model.tar.gz
import tarfile

model_dir = Path('model')
model_dir.mkdir(exist_ok=True)

(model_dir / 'gtzan_cnn.h5').write_bytes((MODEL_SRC / 'gtzan_cnn.h5').read_bytes())
(model_dir / 'mfcc_mean.npy').write_bytes((MODEL_SRC / 'mfcc_mean.npy').read_bytes())
(model_dir / 'mfcc_std.npy').write_bytes((MODEL_SRC / 'mfcc_std.npy').read_bytes())
(model_dir / 'classes.npy').write_bytes((PROCESSED_DIR / 'classes.npy').read_bytes())

with tarfile.open(MODEL_TAR, 'w:gz') as tar:
    tar.add(model_dir, arcname='.')

print('Wrote', MODEL_TAR)


In [ ]:
# Upload model.tar.gz to S3
import sagemaker

sess = sagemaker.Session()
bucket = sess.default_bucket()
prefix = 'gtzan-cnn'
model_s3 = sess.upload_data(str(MODEL_TAR), bucket=bucket, key_prefix=prefix)
print(model_s3)


In [ ]:
# Deploy real-time endpoint
import sagemaker
from sagemaker.tensorflow import TensorFlowModel

role = sagemaker.get_execution_role()

model = TensorFlowModel(
    model_data=model_s3,
    role=role,
    framework_version='2.13',
    py_version='py310',
    entry_point='inference.py',
    source_dir='.',
)

predictor = model.deploy(
    initial_instance_count=1,
    instance_type='ml.m5.large',
)

print('Endpoint:', predictor.endpoint_name)


In [ ]:
# Invoke endpoint with a WAV file
import boto3

runtime = boto3.client('sagemaker-runtime')
with open('sample.wav', 'rb') as f:
    payload = f.read()

resp = runtime.invoke_endpoint(
    EndpointName=predictor.endpoint_name,
    ContentType='audio/wav',
    Body=payload,
)

print(resp['Body'].read().decode('utf-8'))


In [ ]:
# Cleanup (optional)
# predictor.delete_endpoint()
